In [1]:
from functions.satellites_ephemeris import SatelliteEphemeris

ephemeris = SatelliteEphemeris()
ephemeris.load_data()
debris_data = ephemeris.get_debris()

GTOC9 database loaded.


In [2]:
(debris_data[0])

array(['0', 21947.64963766505, 7165739.682446478, 0.0014872294030037296,
       1.7084949743764815, 5.42514944776991, 0.5189646619147478,
       3.220888356932889], dtype=object)

In [3]:
from functions.satellites_ephemeris import SatelliteEphemeris

filename = 'functions/ephemerides/OneWeb_Sat_Data.xlsx'
ephemeris = SatelliteEphemeris(filename=filename)
ephemeris.load_data()
debris_data = ephemeris.get_debris()

Custom database loaded.


In [4]:
(debris_data[0])
debris_data[0][0] = '0'

debris_data[0]

array(['0', 8487.725509259384, 7575901.584828013, 0.0001574,
       1.534114741905732, 1.7968600981594622, 2.2326373964559103,
       6.186628456971504], dtype=object)

In [5]:
t1 = 8487.72551337956
id1 = '0'

t2 = 8492.72551337956
id2 = '1'

cart1 = ephemeris.eph_satellites( t1 * 86400, id1 )
cart2 = ephemeris.eph_satellites( t2 * 86400, id2 )

In [6]:
print(cart1)
print(cart2)

[ 6.81353729e+05 -4.00880032e+06  6.39074168e+06  1.51179714e+03
 -5.93701399e+03 -3.88549629e+03]
[-2.15442729e+05  2.29773839e+06 -7.21763221e+06 -1.50960070e+03
  6.74558714e+03  2.19283395e+03]


In [7]:
cart1[4]

-5937.0139860381705

In [10]:
from functions.MRPLP_J2_analytic import MultiRevolutionPerturbedLambertSolver
from functions.expansion_perturbed_lambert import ExpansionPerturbedLambert
import numpy as np

rr1=cart1[0:3]
vv1=cart1[3:]
rr2=cart2[0:3]
vv2=cart2[3:]
vv1g = vv1

tof = ( t2 - t1 ) * 86400

mu = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

MRPLPsolver = MultiRevolutionPerturbedLambertSolver() # MRPLP solver
EXPpertLamb = ExpansionPerturbedLambert() # Expansion of perturbed Lambert

# set the parameters for the MRPLP solver
order = 7
parameters = MRPLPsolver.mrplp_J2_analytic_parameters(rr1, rr2, tof, vv1g, mu, Re, J2,
                                            order, tol=1.0e-6, cont=0.0, dcontMin=0.1, scl=1.0e-3, itermax=200 )

# solve the MRPLP
output = MRPLPsolver.mrplp_J2_analytic(parameters)

# extract the solution
vv1Sol = output.vv1Sol
vv2Sol = output.vv2Sol
rr2DA = output.rr2DA

# compute the DV
dvv1 = vv1Sol - vv1
dvv2 = vv2 - vv2Sol
dv1 = np.linalg.norm( dvv1 )
dv2 = np.linalg.norm( dvv2 )
dvtot = dv1 + dv2

# print the output
print(f"-------------------------------------------------------")
print(f"                 OUTPUT SUMMARY")
print(f"-------------------------------------------------------")
print(f"Order of the expansion  : {order}")
print(f"Success                 : {output.success}")
print(f"Elapsed time            : {output.elapsed_time} seconds")
print(f"Final pos. error (norm) : {np.linalg.norm( output.rr2DA - rr2 )} m")
print(f"-------------------------------------------------------")
print(f"Delta_v1                : {dv1} m/s")
print(f"Delta_v2                : {dv2} m/s")
print(f"Delta_vtot              : {dvtot} m/s")
print(f"-------------------------------------------------------")

# propagate and obtain the STM
states, vecttof, STM = (
    MultiRevolutionPerturbedLambertSolver.propagatePerturbedKeplerSTM(
        rr1, vv1Sol, tof, 1, parameters
    )
)

-------------------------------------------------------
                 OUTPUT SUMMARY
-------------------------------------------------------
Order of the expansion  : 7
Success                 : True
Elapsed time            : 0.29251861572265625 seconds
Final pos. error (norm) : 2.1105781332667353e-06 m
-------------------------------------------------------
Delta_v1                : 22.71146522768374 m/s
Delta_v2                : 21.014510169245128 m/s
Delta_vtot              : 43.72597539692887 m/s
-------------------------------------------------------


In [11]:
np.linalg.norm(states[-1,0:3] - cart2[0:3])

1.1520571430794717e-07

In [29]:
from functions.analytic_approx_dv_j2 import AnalyticDVApproxJ2
import numpy as np

mue = 398600.4418e9  # [m^3/s^2]
Re = 6378137  # [m]
J2 = 1.08262668e-3

# cart1 = np.array( [ 680792.001246792    ,     -4006594.01935798     ,     6392185.01857288     ,     1512.02932728527      ,   -5938.37973480106, -3883.31816604769 ] )
# cart2 = np.array([-214841.275555147    ,      2295050.57239102    ,     -7218505.37654683    ,     -1509.67931990184    ,       6746.4253028261, 2190.19935704532 ])

# Initialize parameters
parameters = AnalyticDVApproxJ2.approx_DV_parameters(
    rr1=cart1[0:3],
    vv1=cart1[3:],
    rr2=cart2[0:3],
    vv2=cart2[3:],
    t1=t1,
    t2=t2,
    mu=mue,
    rE=Re,
    J2=J2
)

delta_v = AnalyticDVApproxJ2.approx_DV(parameters)
delta_v

1.843308755664876

In [ ]:
from functions.analytic_approx_dv_j2 import *



param = approx_DV_parameters()